In [1]:
import requests
import pandas as pd
from datetime import datetime
import os
from bs4 import BeautifulSoup
import unicodedata

# Search for the data and download

In [2]:
# Configuration

# assume landing page url does not change
landing_page_url = "https://mk.gov.cz/evidence-knihoven-adresar-knihoven-evidovanych-ministerstvem-kultury-a-souvisejici-informace-cs-341"
download_folder = "data"  # name of the folder can be adjusted

os.makedirs(download_folder, exist_ok=True)

def find_xlsx_file_url(html_content, base_url):
    ''' Finds the url of the xlsx file. The expectation is that the first .xlsx
    file link on the webpage is the file containing the list of libraries.'''

    print("Parsing the landing page to find the XLSX file link...")
    soup = BeautifulSoup(html_content, 'html.parser')

    # Look for any 'a' tag with an href ending in .xlsx
    xlsx_link = soup.find('a', href=lambda href: href and href.endswith('.xlsx'))

    if xlsx_link:
        file_href = xlsx_link['href']
        # Check if the URL is absolute or relative
        if file_href.startswith('http'):
            full_file_url = file_href
        else:
            # Construct the full URL for relative links
            full_file_url = base_url + file_href
        print(f"Found file URL: {full_file_url}")
        return full_file_url

    raise Exception("Could not find a link to an .xlsx file on the landing page.")

def download_file(file_url):
    '''Downloads the .xlsx file and saves it.'''
    filename = file_url.split(sep='/')[-1]
    new_filename = os.path.join(download_folder, filename)
    response = requests.get(file_url)
    response.raise_for_status()
    # check if data already exists in folder
    if os.path.exists(new_filename):
        print(f"File already exists at: {new_filename}")
        return new_filename
    # save the data
    with open(new_filename, 'wb') as f:
        f.write(response.content)
    print(f"File is saved at: {new_filename}")
    return new_filename

In [3]:
# Download and save the file
try:
    print(f"Downloading landing page: {landing_page_url}")
    page_response = requests.get(landing_page_url)
    page_response.raise_for_status()

    # Find the file URL from the landing page content
    file_url_to_download = find_xlsx_file_url(page_response.content, "https://mk.gov.cz")

    # Download the actual file
    file = download_file(file_url_to_download)

except Exception as e:
    result_message = f"ERROR - {e}"

Parsing the landing page to find the XLSX file link...
Found file URL: https://mk.gov.cz/doc/cms_library/evidence-knihoven-06082025-20693.xlsx
File already exists at: data/evidence-knihoven-06082025-20693.xlsx


# Data Preprocessing



## First look at the data

In [4]:
# load the data into a pandas dataframe
df = pd.read_excel(file)

/usr/local/lib/python3.12/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: Tabulka1[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [5]:
# first glance at the data
df.head()

,Název provozovatele,B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno),C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - obec,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - okres,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - kraj,"C/CH - datová schránka, má-li ji subjekt zřízenou",D - předmět činnosti,E - právní forma,...,T - datum evidence knihovny,XXT - schvalovatel záznamu,U - datum aktualizace záznamu,XXU - schvalovatel aktualizace záznamu,aktivní / zrušená (vyřazená z evidence),V - datum vyřazení,XXV - vyřadil z evidence,W - důvod vyřazení z evidence,Y - číslo jednací,Poznámka
0,Obec Kralice na Hané,288390,Masarykovo náměstí 41,798 12,Kralice na Hané,Prostějov,Olomoucký,NaN,obecní úřad,NaN,...,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN,,NaN,k 17457/2001,NaN
1,Obec Lukavice,270431,Lukavice,538 21,Slatiňany,Chrudim,Pardubický,NaN,obecní úřad,NaN,...,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN,,NaN,k 205984/2001,NaN
2,Muzeum Beskyd Frýdek-Místek,95630,Hluboká 66,738 01,Frýdek-Místek,Frýdek-Místek,Moravskoslezský,NaN,"uchování, zpracování a zpřístupňování sbírek m...",NaN,...,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN,,NaN,k 20588/2001,NaN
3,Městská kulturní zařízení v Bělé pod Bezdězem,62451502,Masarykovo náměstí 140,294 21,Bělá pod Bezdězem,Mladá Boleslav,Středočeský,NaN,zajišťování kulturních hodnot a aktivit,NaN,...,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN,,NaN,k 606/2002,NaN
4,Město Loket,259489,T. G. Masaryka 1,357 33,Loket,Sokolov,Karlovarský,NaN,městský úřad,NaN,...,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN,,NaN,k 499/2002,NaN


In [6]:
df.iloc[:,9:27].head()  # check the rest of the columns

,E - právní forma,G - datum narození provozovatele - fyzické osoby,I - NÁZEV KNIHOVNY,R - EVIDENČNÍ ČÍSLO KNIHOVNY,J - druh knihovny,Kategorie - pomocný sloupec,K - adresa knihovny: ulice,K - adresa knihovny: PSČ,K - adresa knihovny: město,K - adresa knihovny: okres,K - adresa knihovny: kraj,M - emailový kontakt na autora záznamu,N - e-mailový kontakt na knihovnu,"O - odkaz na webovou stránku knihovny, respektive odkaz na informace o knihovně na webových stránkách provozovatele",P - provozní doba knihovny,Q - další informace,X - sigla,S - datum vytvoření záznamu
0,NaN,NaN,Základní knihovna v Kralicích na Hané,1,základní,public,Masarykovo náměstí 41,798 12,Kralice na Hané,Prostějov,Olomoucký,NaN,,NaN,,NaN,,2003-05-23
1,NaN,NaN,Základní knihovna v Lukavici,2,základní,public,Lukavice 1,538 21,Slatiňany,Chrudim,Pardubický,NaN,,NaN,,NaN,,2003-05-23
2,NaN,NaN,Studijní knihovna Muzea Beskyd Frýdek - Místek,3,základní se specializovaným knihovním fondem,muzejní,Hluboká 66,738 01,Frýdek-Místek,Frýdek-Místek,Moravskoslezský,NaN,muzeumbeskyd@telecom.cz,NaN,"9.00-11.00, 13.00-14.30 - 9.00-11.00, 13.00-...",NaN,FMR 301,2003-05-23
3,NaN,NaN,Městská knihovna Vladimíra Holana,4,základní,public,Masarykovo náměstí 140,294 21,Bělá pod Bezdězem,Mladá Boleslav,Středočeský,NaN,mekz@knihovnabelapb.cz,NaN,,NaN,MBG 505,2003-05-23
4,NaN,NaN,Městská knihovna Loket,5,základní,public,T. G. Masaryka 96,357 33,Loket,Sokolov,Karlovarský,NaN,,http://www.mkloket.cz,,NaN,SOG501,2003-05-23


In [7]:
print("\nData shape:", df.shape)


Data shape: (6716, 37)


In [8]:
# Basic info, including column names, dtype, non-null counts
print(df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6716 entries, 0 to 6715
Data columns (total 37 columns):
 #   Column                                                                                                               Non-Null Count  Dtype         
---  ------                                                                                                               --------------  -----         
 0   Název provozovatele                                                                                                  6716 non-null   object        
 1   B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno)                                                        6716 non-null   int64         
 2   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice                                                        6716 non-null   object        
 3   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ                                                          6716 n

## Removing empty columns

In [9]:
print(df.iloc[:,10].unique())  # the 10th col has no values other than NaN, we can remove it

[nan]


In [10]:
df = df.drop(df.columns[10], axis=1)

In [11]:
# we can leave these columns, as they do contain some data
print(df.iloc[:,23].unique())
print(df.iloc[:,33].unique())

[nan
 'Dočasná adresa knihovny do roku 2026: Žižkovo náměstí 10, 390 01 Tábor; korespondenční adresa: Jiráskova 1775, 390 01 Tábor'
 'KOHA']
[nan 'zrušení knihovny' 'žádost starostky obce'
 'org. důvody sloučení knihoven' 'knihovna zrušena']


In [12]:
df[df.isna().all(axis=1)] # no empty rows

,Název provozovatele,B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno),C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - obec,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - okres,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - kraj,"C/CH - datová schránka, má-li ji subjekt zřízenou",D - předmět činnosti,E - právní forma,...,T - datum evidence knihovny,XXT - schvalovatel záznamu,U - datum aktualizace záznamu,XXU - schvalovatel aktualizace záznamu,aktivní / zrušená (vyřazená z evidence),V - datum vyřazení,XXV - vyřadil z evidence,W - důvod vyřazení z evidence,Y - číslo jednací,Poznámka


## Apply datetime format to columns


In [13]:
print(df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6716 entries, 0 to 6715
Data columns (total 36 columns):
 #   Column                                                                                                               Non-Null Count  Dtype         
---  ------                                                                                                               --------------  -----         
 0   Název provozovatele                                                                                                  6716 non-null   object        
 1   B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno)                                                        6716 non-null   int64         
 2   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice                                                        6716 non-null   object        
 3   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ                                                          6716 n

In [14]:
df.iloc[:, 21:32].head()  # the columns containing dates are S, T, U, V - indexes 25, 26, 28, 31

,"O - odkaz na webovou stránku knihovny, respektive odkaz na informace o knihovně na webových stránkách provozovatele",P - provozní doba knihovny,Q - další informace,X - sigla,S - datum vytvoření záznamu,T - datum evidence knihovny,XXT - schvalovatel záznamu,U - datum aktualizace záznamu,XXU - schvalovatel aktualizace záznamu,aktivní / zrušená (vyřazená z evidence),V - datum vyřazení
0,NaN,,NaN,,2003-05-23,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN
1,NaN,,NaN,,2003-05-23,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN
2,NaN,"9.00-11.00, 13.00-14.30 - 9.00-11.00, 13.00-...",NaN,FMR 301,2003-05-23,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN
3,NaN,,NaN,MBG 505,2003-05-23,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN
4,http://www.mkloket.cz,,NaN,SOG501,2003-05-23,2002-07-15 00:00:00,Čermáková,NaN,NaN,A,NaN


In [15]:
df.iloc[:,25].unique()  # check the only column in datetime format

<DatetimeArray>
['2003-05-23 00:00:00', '2003-05-27 00:00:00', '2003-05-28 00:00:00',
 '2002-07-15 00:00:00', '2003-05-29 00:00:00', '2002-05-29 00:00:00',
 '2003-05-30 00:00:00', '2003-06-02 00:00:00', '2004-09-02 00:00:00',
 '2003-06-03 00:00:00',
 ...
 '2024-08-29 00:00:00', '2024-09-03 00:00:00', '2024-10-23 00:00:00',
 '2024-11-11 00:00:00', '2025-01-06 00:00:00', '2025-01-20 00:00:00',
 '2025-02-27 00:00:00', '2025-03-03 00:00:00', '2025-05-15 00:00:00',
 '2025-06-12 00:00:00']
Length: 473, dtype: datetime64[ns]

### Column 26

In [16]:
# let's change the other columns containing date time information into datetime format
(df.iloc[:,26].unique())
# some rows contain more than one date, which makes conversion impossible
# we will replace these rows with the most recent date in the record, and then retype to datetime format

mask = df.iloc[:,26].astype(str).str.contains(", ")
new_values = df[mask].iloc[:,26].astype(str).str.split(', ', expand=True)[1]
df.iloc[new_values.index, 26] = new_values
new_values

,1
6237,27.6.2022
6347,15.5.2022


In [17]:
# change the format of column 26
df[df.columns[26]] = pd.to_datetime(df[df.columns[26]])

### Column 28

In [18]:
# column 28
df.iloc[:,28].unique()
# again some rows contain more than one date, which makes conversion impossible


array([nan, datetime.datetime(2022, 5, 23, 0, 0),
       datetime.datetime(2008, 4, 4, 0, 0),
       datetime.datetime(2025, 7, 7, 0, 0),
       datetime.datetime(2007, 7, 13, 0, 0),
       datetime.datetime(2004, 3, 10, 0, 0),
       datetime.datetime(2006, 3, 10, 0, 0),
       datetime.datetime(2007, 11, 29, 0, 0),
       datetime.datetime(2006, 6, 21, 0, 0), '9.2.2008, 6.1.2020',
       datetime.datetime(2023, 11, 8, 0, 0),
       datetime.datetime(2022, 7, 26, 0, 0), '23.05.2022, 15.03.2024',
       datetime.datetime(2017, 6, 1, 0, 0),
       datetime.datetime(2022, 3, 17, 0, 0),
       datetime.datetime(2012, 3, 19, 0, 0),
       datetime.datetime(2007, 5, 26, 0, 0),
       datetime.datetime(2008, 1, 4, 0, 0), '9.1.2007, 12.11.2018',
       datetime.datetime(2015, 11, 10, 0, 0),
       datetime.datetime(2008, 3, 27, 0, 0), '11.2.2009, 26.1.2017',
       datetime.datetime(2015, 4, 24, 0, 0),
       datetime.datetime(2018, 11, 12, 0, 0),
       datetime.datetime(2016, 4, 19, 0, 0),


In [19]:
# replace these rows with the most recent date in the record, and then retype to datetime format
mask = df.iloc[:,28].astype(str).str.contains(",|;")
table = df[mask].iloc[:,28].astype(str).str.split(',|;', expand=True)
# rows 1800 and 5333 need to be corrected, there are some errors
print(df.iloc[[1800,5333], 28])
df.iloc[1800, 28] = df.iloc[1800, 28].replace(',', '.')
df.iloc[5333, 28] = df.iloc[5333, 28].split(',')[1][7:]
df.iloc[[1800,5333], 28]

1800                       4,4.2016
5333    19.2.2007, AD542212.11.2018
Name: U - datum aktualizace záznamu, dtype: object


,U - datum aktualizace záznamu
1800,4.4.2016
5333,12.11.2018


In [20]:
# replace by the most recent date
mask = df.iloc[:,28].astype(str).str.contains(",|;")
dates = df[mask].iloc[:,28].astype(str).str.split(',|;', expand=True)
dates = dates.apply(lambda x: x.dropna().iloc[-1] if not x.dropna().empty else None, axis=1)
df.loc[mask, df.columns[28]] = dates
df.loc[mask, df.columns[28]]

,U - datum aktualizace záznamu
43,6.1.2020
52,15.03.2024
83,12.11.2018
96,26.1.2017
249,19.11.2020
...,...
5929,23.5.2022
6184,11.04.2024
6340,27.7.2023
6413,22.05.2025


In [21]:
# there is some leftover text in one of the records, we can remove it also
df[df.iloc[:, 28].astype(str).str.contains('oprava dat|oprava datum', case=False, na=False)]
ind = df[df.iloc[:, 28].astype(str).str.contains('oprava dat|oprava datum', case=False, na=False)].index
df.iloc[ind, 28] = df.iloc[ind, 28].astype(str).str.replace(r'\s*oprava dat\s*', '', regex=True).str.replace(r'\s*oprava datum\s*', '', regex=True)

In [22]:
# convert to datetime
df[df.columns[28]] = pd.to_datetime(df[df.columns[28]])

### Column 31

In [23]:
# again there are rows containing more than one date.
# the column contains the date of record deletion. Again it would be possible to choose the most recent date and convert.
(df.iloc[:,31].unique())
pd.to_datetime(df.iloc[:,31])

DateParseError: Unknown datetime string format, unable to parse: 15.09.2022,3.10.2022, at position 113

In [24]:
mask = df.iloc[:,31].astype(str).str.contains(",|;")
df[mask].iloc[:,31].astype(str).str.split(',|;', expand=True)

,0,1
2523,15.09.2022,3.10.2022


In [25]:
# just one record is inconsistent - again we will replace by the most recent date
val = df[mask].iloc[:,31].astype(str).str.split(',|;', expand=True)[1]  # only one record has more dates
ind = df[mask].index
df.iloc[ind, 31] = val
(df.iloc[ind, 31])

,V - datum vyřazení
2523,3.10.2022


In [26]:
# the day and month number do not match:
df[df.columns[31]] = pd.to_datetime(df[df.columns[31]])

ValueError: day is out of range for month, at position 152

In [27]:
df[df.iloc[:,31] == '31.9.2013']  # September has just 30 days - correct to 30.9.2013
df.loc[df.iloc[:,31] == '31.9.2013', df.columns[31]] = '30.9.2013'
df[df.iloc[:,31] == '31.9.2013']

,Název provozovatele,B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno),C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - obec,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - okres,C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - kraj,"C/CH - datová schránka, má-li ji subjekt zřízenou",D - předmět činnosti,E - právní forma,...,T - datum evidence knihovny,XXT - schvalovatel záznamu,U - datum aktualizace záznamu,XXU - schvalovatel aktualizace záznamu,aktivní / zrušená (vyřazená z evidence),V - datum vyřazení,XXV - vyřadil z evidence,W - důvod vyřazení z evidence,Y - číslo jednací,Poznámka


In [28]:
# change dtype to datetime
df[df.columns[31]] = pd.to_datetime(df[df.columns[31]])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6716 entries, 0 to 6715
Data columns (total 36 columns):
 #   Column                                                                                                               Non-Null Count  Dtype         
---  ------                                                                                                               --------------  -----         
 0   Název provozovatele                                                                                                  6716 non-null   object        
 1   B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno)                                                        6716 non-null   int64         
 2   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice                                                        6716 non-null   object        
 3   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ                                                          6716 n

## Apply string dtype to columns

In [29]:
(df.iloc[:,[1,11]])  # 11. col can possibly be removed, it's the index number - 1
df.iloc[:,11].unique()

array([   1,    2,    3, ..., 6714, 6715, 6716])

In [30]:
# change dtype to string
cols = df.select_dtypes(include=['object']).columns
df[cols] = df[cols].astype('string')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6716 entries, 0 to 6715
Data columns (total 36 columns):
 #   Column                                                                                                               Non-Null Count  Dtype         
---  ------                                                                                                               --------------  -----         
 0   Název provozovatele                                                                                                  6716 non-null   string        
 1   B/H - IČ provozovatele (u fyzické osoby pokud bylo přiděleno)                                                        6716 non-null   int64         
 2   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - ulice                                                        6716 non-null   string        
 3   C/CH - sídlo/adresa; místo trvalého pobytu fyz. osoby - PSČ                                                          6716 n

## Adjust column names

In [31]:
# remove Slavic diacritics to ensure interoperability

def normalise(name: str) -> str:
  name = str(name)
  name = unicodedata.normalize('NFKD', name)
  name = ''.join([c for c in name if not unicodedata.combining(c)])
  return name

original_col_names = df.columns  # save original column names
df.columns = df.columns.map(lambda x: normalise(x))  #.lower().replace('-', '').replace('  ', '_').replace(' ', '_'))

# it can also be useful to shorten the column names, remove the whitespace, ...

## Save the data

In [32]:
df.to_csv('cleaned_data.csv', index=False)